In [1]:
import os
from PyPDF2 import PdfReader

pdf_folder = "/Users/anandnishad/Desktop/harry_potter_rag"

pdf_files = sorted([f for f in os.listdir(pdf_folder) if f.endswith(".pdf")])

# Initialize an empty string to store the combined text
text = ""
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)
    reader = PdfReader(pdf_path)

    print(f"Reading: {pdf_file}")

    for page in reader.pages:
        text_1 = page.extract_text()
        if text_1:
            text += text_1 + "\n\n" 
    # print(text)


In [20]:
# print(text)
def chunk_text(text, chunk_size=500):
    words = text.split()
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]
    return chunks

text_chunks = chunk_text(text, chunk_size=500)

In [21]:
# print(text_chunks)


In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(text_chunks)  # text_chunks is a list of split text


In [23]:
embedding_dim = embeddings.shape[1] 
print(embedding_dim)

384


In [24]:
import faiss

index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings)

In [25]:
from sentence_transformers import SentenceTransformer

# Load the same embedding model used earlier
model = SentenceTransformer('all-MiniLM-L6-v2')

# Convert query to an embedding
query = "Who killed Albus Dumbledore"
query_embedding = model.encode([query])


In [19]:
import numpy as np

# Number of closest chunks to retrieve
k = 5000  

# Perform similarity search
distances, indices = index.search(np.array(query_embedding), k)

# Retrieve relevant text chunks
retrieved_chunks = [text_chunks[i] for i in indices[0]]

print("Most Relevant Chunk:")
print(retrieved_chunks[0])  # The most relevant text


Most Relevant Chunk:
had me use any wizard, would you not, Wormtail? Any wizard who had hated me . . . as so many of them still do. But I knew the one I must use, if I was to rise again, more powerful than I had been when I had fallen. I wanted Harry Potter’s blood. I wanted the blood of the one who had stripped me of power thirteen years ago . . . for the lingering protection his mother once gave him would then reside in my veins too. . . . “But how to get at Harry Potter? For


In [ ]:
import openai


def construct_prompt(input_text, refrence_data):

    prompt = f"""the refrence data {refrence_data} my query {input_text}  read refrence_data and answer query form refrence data only esle return insufficent data

    """



    return prompt


def chat_with_gpt(input_text, group_subgroup_json):
    
    openai.api_key = ""


    # Create the prompt
    prompt = construct_prompt(input_text, group_subgroup_json)

    # Call the API
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo-2024-04-09",
        messages=[
            {"role": "system", "content": prompt}
        ],
        temperature=0,
        top_p=1,
    )

    # Extract and return the response content
    usage = response["usage"]
    return response.choices[0].message.content

In [1]:
data = chat_with_gpt(query,retrieved_chunks[0])
print(data)

Snape Killed Albus Dumbledore


In [ ]:
pip install openai==0.28